# import packages

In [1]:
import pandas as pd

from sklearn.compose import make_column_selector, ColumnTransformer, make_column_transformer
# sklearn.compose: The sklearn.compose module is a submodule of the sklearn library for machine learning in Python. It provides functions for creating complex preprocessing and modeling pipelines.
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures,RobustScaler
#sklearn.preprocessing: The sklearn.preprocessing module is a submodule of the sklearn library that provides functions for preprocessing data, such as scaling and normalizing features, imputing missing values, and encoding categorical variables.
from sklearn.linear_model import Ridge,LinearRegression,Lasso, ElasticNet
# sklearn.linear_model: The sklearn.linear_model module is a submodule of the sklearn library that provides functions for fitting linear models for regression and classification.
from sklearn.pipeline import make_pipeline
# sklearn.pipeline: The sklearn.pipeline module is a submodule of the sklearn library that provides functions for creating and working with pipelines of transformers and models.
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve, RandomizedSearchCV, cross_val_score, KFold
# sklearn.model_selection: The sklearn.model_selection module is a submodule of the sklearn library that provides functions for splitting data into training and test sets, evaluating models using cross-validation, and hyperparameter tuning.
from sklearn.dummy import DummyRegressor
# sklearn.dummy: The sklearn.dummy module is a submodule of the sklearn library that provides simple dummy models for regression and classification.


from sklearn.impute import SimpleImputer
import numpy as np

from sklearn.ensemble import RandomForestClassifier


# Read data

In [2]:
try:
    df = pd.read_csv('data/data_110k_lignes.csv')
except:
    print("Error: no such file csv")


In [3]:
df['explicit'] = df['explicit'].apply(lambda x : 0 if x == False else 1)
df['is_local'] = df['is_local'].apply(lambda x : 0 if x == False else 1)
del df['is_local']

# Rmove "Unnamed: 0" column

In [4]:
try:
    df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("No such column")

# Remove duplicates

In [5]:
df = df.drop_duplicates()



# Préparation à la modélisation

In [6]:
X = df.drop('popularity', axis=1)
y = df['popularity']

In [7]:
X = X[[col for col in X.columns if X[col].dtype != 'object' or col == 'genre']]

In [8]:
def make_pipeline_to_ML(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=42)
    numerical_features = make_column_selector(dtype_include=np.number)
    categorical_features = make_column_selector(dtype_exclude= np.number)
    numerical_pipeline = make_pipeline(StandardScaler(with_mean=False))
    categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore'))
    preprocessor = make_pipeline(make_column_transformer((numerical_pipeline, numerical_features),
                                    (categorical_pipeline, categorical_features)),PolynomialFeatures(2)
                                    )
    return preprocessor, X_train, X_test, y_train, y_test


# Create the pipeline
preprocessor, X_train, X_test, y_train, y_test = make_pipeline_to_ML(X,y)

In [9]:
lasso_model = make_pipeline(preprocessor, Lasso(random_state=42))
params = {
        'lasso__alpha':[0.1,1,10], 
        }
 # Create the grid search object
grid_seargrid_search_lasso = GridSearchCV(lasso_model, params, cv=5, n_jobs = -1, verbose=4)

In [10]:
# Fit the grid search to the data
grid_seargrid_search_lasso.fit(X_train, y_train)
# Print the best parameters and the best score
print("Best parameters: ", grid_seargrid_search_lasso.best_params_)
print("Best score: ", grid_seargrid_search_lasso.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 2/5] END ....................lasso__alpha=1;, score=0.412 total time= 1.9min
[CV 1/5] END ....................lasso__alpha=1;, score=0.413 total time= 1.9min
[CV 3/5] END ....................lasso__alpha=1;, score=0.412 total time= 2.2min
[CV 1/5] END ...................lasso__alpha=10;, score=0.124 total time=  34.3s
[CV 2/5] END ...................lasso__alpha=10;, score=0.123 total time=  35.4s
[CV 4/5] END ....................lasso__alpha=1;, score=0.413 total time= 1.7min
[CV 4/5] END ..................lasso__alpha=0.1;, score=0.703 total time= 3.7min
[CV 5/5] END ....................lasso__alpha=1;, score=0.417 total time= 1.9min
[CV 4/5] END ...................lasso__alpha=10;, score=0.121 total time=  30.3s
[CV 3/5] END ...................lasso__alpha=10;, score=0.123 total time=  40.7s
[CV 5/5] END ...................lasso__alpha=10;, score=0.126 total time=  34.8s
[CV 2/5] END ..................lasso__alpha=0.1;,

/home/apprenant/miniconda3/envs/spotify/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 46372.25463058241, tolerance: 2388.190491297232
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 3/5] END ..................lasso__alpha=0.1;, score=0.703 total time= 4.6min


/home/apprenant/miniconda3/envs/spotify/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61908.32952390518, tolerance: 2389.291803250171
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 5/5] END ..................lasso__alpha=0.1;, score=0.700 total time= 4.8min


/home/apprenant/miniconda3/envs/spotify/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 48955.61663639918, tolerance: 2376.9283783845713
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 1/5] END ..................lasso__alpha=0.1;, score=0.714 total time= 4.9min
Best parameters:  {'lasso__alpha': 0.1}
Best score:  0.7058474950925459


/home/apprenant/miniconda3/envs/spotify/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74017.57627941296, tolerance: 2982.775607649173
  model = cd_fast.sparse_enet_coordinate_descent(
